# Gender Bias in Large Language Models - Comprehensive Analysis

This notebook provides a complete analysis framework for the Gender Bias in LLMs study. It includes data loading, statistical analysis, visualization, and interpretation of results.

## Study Overview

This research investigates how different prompting strategies affect gender bias in LLM outputs:

1. **Raw Prompt** (Control) - Basic rewrite request
2. **System Prompt** - Explicit gender-neutral instructions  
3. **Few-Shot** - Examples + instructions
4. **Few-Shot + Verification** - Examples + self-verification

### Evaluation Metrics

- **Gender Bias Score** - Automated detection of gendered terms
- **Fluency Score** - Text quality assessment
- **BLEU-4 Score** - Meaning preservation
- **Semantic Similarity** - Content preservation

---

## 1. Import Required Libraries

Setting up the analysis environment with all necessary packages.

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import json
import sys
import os
from pathlib import Path
import time
import re
from datetime import datetime
from typing import Dict, List, Any, Optional

# Statistical analysis
import scipy.stats as stats
from scipy.stats import f_oneway, ttest_ind
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Progress tracking
from tqdm.notebook import tqdm
from IPython.display import display, HTML, Markdown

# Set up plotting
plt.style.use('default')
sns.set_palette("Set2")
pio.templates.default = "plotly_white"

# Add project root to path
project_root = Path.cwd().parent
sys.path.append(str(project_root / "src"))

print("✓ All libraries imported successfully!")
print(f"Working directory: {Path.cwd()}")
print(f"Project root: {project_root}")

## 2. Data Loading and Exploration

Loading experiment results and exploring the dataset structure.

In [ ]:
# Find the most recent results file
results_dir = project_root / "data" / "results"
results_files = list(results_dir.glob("experiment_results_*.json"))

if not results_files:
    print("❌ No experiment results found!")
    print(f"Please run the experiment first: python {project_root}/main.py run-experiment")
else:
    # Load most recent results
    latest_results_file = max(results_files, key=lambda f: f.stat().st_mtime)
    print(f"📁 Loading results from: {latest_results_file.name}")
    
    with open(latest_results_file, 'r') as f:
        results_data = json.load(f)
    
    print(f"✓ Loaded experiment data:")
    print(f"  - Experiment ID: {results_data['experiment_id']}")
    print(f"  - Status: {results_data['status']}")
    print(f"  - Total experiments: {results_data['total_experiments']}")
    print(f"  - Timestamp: {results_data['timestamp']}")
    
    # Show configuration
    config = results_data['configuration']
    print(f"\n📋 Experiment Configuration:")
    print(f"  - Repetitions per paragraph: {config['repetitions_per_paragraph']}")
    print(f"  - Strategies: {', '.join(config['prompt_strategies'])}")
    print(f"  - Models: {', '.join(config['llm_models'])}")
    print(f"  - Temperature: {config['temperature']}")

In [ ]:
# Convert results to DataFrame for analysis
def create_dataframe_from_results(results_data):
    """Convert experiment results to pandas DataFrame"""
    rows = []
    
    for result in results_data.get("detailed_results", []):
        if result.get("success", False):
            evaluation = result["evaluation"]
            summary_scores = evaluation["summary_scores"]
            
            row = {
                "experiment_id": result["experiment_id"],
                "paragraph_id": result["paragraph_id"],
                "strategy": result["strategy"],
                "model": result["model"],
                "repetition": result["repetition"],
                "bias_reduction_percentage": summary_scores["bias_reduction_percentage"],
                "is_gender_neutral": summary_scores["is_gender_neutral"],
                "fluency_score": summary_scores["fluency_score"],
                "bleu_4_score": summary_scores["bleu_4_score"],
                "semantic_similarity": summary_scores["semantic_similarity"],
                "generation_time": result["generation_time"],
                
                # Additional metrics
                "original_bias_score": evaluation["bias_evaluation"]["original_bias"]["bias_score"],
                "generated_bias_score": evaluation["bias_evaluation"]["generated_bias"]["bias_score"],
                "total_gendered_terms_original": evaluation["bias_evaluation"]["original_bias"]["total_gendered_terms"],
                "total_gendered_terms_generated": evaluation["bias_evaluation"]["generated_bias"]["total_gendered_terms"],
            }
            rows.append(row)
    
    return pd.DataFrame(rows)

# Create DataFrame
if 'results_data' in locals():
    df = create_dataframe_from_results(results_data)
    
    print(f"📊 DataFrame created with {len(df)} rows and {len(df.columns)} columns")
    print(f"\n🔍 Data Overview:")
    print(f"  - Strategies: {df['strategy'].unique()}")
    print(f"  - Models: {df['model'].unique()}")
    print(f"  - Paragraphs: {df['paragraph_id'].nunique()}")
    print(f"  - Total repetitions: {df['repetition'].max()}")
    
    # Display first few rows
    print(f"\n📋 Sample Data:")
    display(df.head())
    
    # Basic statistics
    print(f"\n📈 Summary Statistics:")
    display(df[['bias_reduction_percentage', 'fluency_score', 'bleu_4_score', 'semantic_similarity']].describe())

## 3. Statistical Analysis

Performing comprehensive statistical analysis including ANOVA tests and post-hoc comparisons.

In [ ]:
def perform_anova_analysis(df, dependent_var, independent_var="strategy"):
    """Perform ANOVA test and return results"""
    
    # Group data by independent variable
    groups = []
    group_names = []
    
    for group_name in df[independent_var].unique():
        group_data = df[df[independent_var] == group_name][dependent_var]
        groups.append(group_data)
        group_names.append(group_name)
    
    # Perform one-way ANOVA
    f_stat, p_value = f_oneway(*groups)
    
    # Calculate effect size (eta-squared)
    ss_between = sum(len(group) * (group.mean() - df[dependent_var].mean())**2 for group in groups)
    ss_total = ((df[dependent_var] - df[dependent_var].mean())**2).sum()
    eta_squared = ss_between / ss_total if ss_total > 0 else 0
    
    # Group statistics
    group_stats = {}
    for i, group_name in enumerate(group_names):
        group_stats[group_name] = {
            "mean": groups[i].mean(),
            "std": groups[i].std(),
            "count": len(groups[i])
        }
    
    return {
        "dependent_variable": dependent_var,
        "f_statistic": f_stat,
        "p_value": p_value,
        "eta_squared": eta_squared,
        "significant": p_value < 0.05,
        "group_statistics": group_stats,
        "groups": groups,
        "group_names": group_names
    }

def perform_post_hoc_tests(df, dependent_var, independent_var="strategy"):
    """Perform post-hoc pairwise comparisons"""
    
    # Use Tukey's HSD test
    tukey_result = pairwise_tukeyhsd(
        endog=df[dependent_var],
        groups=df[independent_var],
        alpha=0.05
    )
    
    return tukey_result

# Perform ANOVA for each metric
if 'df' in locals():
    metrics = ["bias_reduction_percentage", "fluency_score", "bleu_4_score", "semantic_similarity"]
    anova_results = {}
    
    print("🔬 Performing ANOVA Analysis")
    print("=" * 50)
    
    for metric in metrics:
        print(f"\n📊 {metric.replace('_', ' ').title()}:")
        
        anova_result = perform_anova_analysis(df, metric)
        anova_results[metric] = anova_result
        
        print(f"  F-statistic: {anova_result['f_statistic']:.4f}")
        print(f"  p-value: {anova_result['p_value']:.6f}")
        print(f"  Effect size (η²): {anova_result['eta_squared']:.4f}")
        
        if anova_result['significant']:
            print(f"  ✅ SIGNIFICANT difference between strategies (p < 0.05)")
            
            # Perform post-hoc tests
            tukey_result = perform_post_hoc_tests(df, metric)
            print(f"  📋 Post-hoc comparisons (Tukey's HSD):")
            print(f"     {tukey_result}")
        else:
            print(f"  ❌ No significant difference between strategies")
        
        # Show group means
        print(f"  📈 Group means:")
        for group, stats in anova_result['group_statistics'].items():
            print(f"     {group}: {stats['mean']:.3f} (±{stats['std']:.3f})")

    print(f"\n✅ ANOVA analysis complete!")

## 4. Comprehensive Visualizations

Creating multiple visualizations to understand the results from different perspectives.

In [ ]:
# 1. Strategy Comparison - Box Plots
if 'df' in locals():
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.ravel()
    
    metrics = ["bias_reduction_percentage", "fluency_score", "bleu_4_score", "semantic_similarity"]
    
    for i, metric in enumerate(metrics):
        sns.boxplot(data=df, x="strategy", y=metric, ax=axes[i])
        axes[i].set_title(f'{metric.replace("_", " ").title()} by Strategy')
        axes[i].tick_params(axis='x', rotation=45)
        
        # Add significance markers if ANOVA was significant
        if 'anova_results' in locals() and anova_results[metric]['significant']:
            axes[i].text(0.02, 0.98, '***', transform=axes[i].transAxes, 
                        fontsize=16, fontweight='bold', va='top', color='red')
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Strategy comparison box plots generated")

In [ ]:
# 2. Interactive Scatter Plot - Trade-off Analysis
if 'df' in locals():
    fig = px.scatter(
        df, 
        x="bias_reduction_percentage", 
        y="fluency_score",
        color="strategy", 
        size="bleu_4_score",
        hover_data=["paragraph_id", "model", "semantic_similarity"],
        title="Trade-off Analysis: Bias Reduction vs Fluency",
        labels={
            "bias_reduction_percentage": "Bias Reduction (%)",
            "fluency_score": "Fluency Score"
        }
    )
    
    fig.update_layout(
        width=800,
        height=600,
        showlegend=True
    )
    
    fig.show()
    
    print("🎯 Interactive trade-off analysis generated")

In [ ]:
# 3. Performance Dashboard
if 'df' in locals():
    # Create subplots
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Mean Bias Reduction by Strategy', 'Success Rate by Strategy',
                       'Quality Metrics by Strategy', 'Performance Distribution'),
        specs=[[{"type": "bar"}, {"type": "bar"}],
               [{"type": "bar"}, {"type": "histogram"}]]
    )
    
    # 1. Mean bias reduction
    strategy_bias = df.groupby("strategy")["bias_reduction_percentage"].mean().reset_index()
    fig.add_trace(
        go.Bar(x=strategy_bias["strategy"], y=strategy_bias["bias_reduction_percentage"],
               name="Bias Reduction", showlegend=False),
        row=1, col=1
    )
    
    # 2. Success rates
    success_rates = df.groupby("strategy")["is_gender_neutral"].mean().reset_index()
    fig.add_trace(
        go.Bar(x=success_rates["strategy"], y=success_rates["is_gender_neutral"],
               name="Success Rate", showlegend=False, marker_color="green"),
        row=1, col=2
    )
    
    # 3. Quality metrics
    quality_metrics = df.groupby("strategy")[["fluency_score", "bleu_4_score"]].mean().reset_index()
    fig.add_trace(
        go.Bar(x=quality_metrics["strategy"], y=quality_metrics["fluency_score"],
               name="Fluency", marker_color="blue"),
        row=2, col=1
    )
    fig.add_trace(
        go.Bar(x=quality_metrics["strategy"], y=quality_metrics["bleu_4_score"],
               name="BLEU-4", marker_color="orange"),
        row=2, col=1
    )
    
    # 4. Distribution
    fig.add_trace(
        go.Histogram(x=df["bias_reduction_percentage"], nbinsx=20,
                    name="Distribution", showlegend=False, marker_color="purple"),
        row=2, col=2
    )
    
    fig.update_layout(
        height=800,
        title_text="Performance Dashboard",
        showlegend=True
    )
    
    fig.show()
    
    print("📈 Performance dashboard generated")

In [ ]:
# 4. Correlation Analysis
if 'df' in locals():
    # Calculate correlation matrix
    metrics = ["bias_reduction_percentage", "fluency_score", "bleu_4_score", 
               "semantic_similarity", "generation_time"]
    corr_matrix = df[metrics].corr()
    
    # Create heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
                square=True, fmt='.3f')
    plt.title('Correlation Matrix of Evaluation Metrics')
    plt.tight_layout()
    plt.show()
    
    print("🔗 Correlation analysis completed")
    
    # Print key correlations
    print("\n🔍 Key Correlations:")
    bias_fluency_corr = corr_matrix.loc['bias_reduction_percentage', 'fluency_score']
    bias_bleu_corr = corr_matrix.loc['bias_reduction_percentage', 'bleu_4_score']
    fluency_bleu_corr = corr_matrix.loc['fluency_score', 'bleu_4_score']
    
    print(f"  Bias Reduction ↔ Fluency: {bias_fluency_corr:.3f}")
    print(f"  Bias Reduction ↔ BLEU-4: {bias_bleu_corr:.3f}")
    print(f"  Fluency ↔ BLEU-4: {fluency_bleu_corr:.3f}")

## 5. Key Findings and Interpretation

Interpreting the statistical results and their implications for gender bias mitigation.

In [ ]:
# Generate comprehensive findings summary
def generate_findings_summary(df, anova_results):
    """Generate a comprehensive summary of findings"""
    
    findings = {
        "strategy_performance": {},
        "statistical_significance": {},
        "best_performers": {},
        "trade_offs": {},
        "success_rates": {}
    }
    
    # Strategy performance
    for strategy in df['strategy'].unique():
        strategy_data = df[df['strategy'] == strategy]
        findings["strategy_performance"][strategy] = {
            "bias_reduction": {
                "mean": strategy_data["bias_reduction_percentage"].mean(),
                "std": strategy_data["bias_reduction_percentage"].std()
            },
            "fluency": {
                "mean": strategy_data["fluency_score"].mean(),
                "std": strategy_data["fluency_score"].std()
            },
            "meaning_preservation": {
                "mean": strategy_data["bleu_4_score"].mean(),
                "std": strategy_data["bleu_4_score"].std()
            },
            "neutralization_success_rate": strategy_data["is_gender_neutral"].mean()
        }
    
    # Statistical significance
    for metric, result in anova_results.items():
        findings["statistical_significance"][metric] = {
            "significant": result["significant"],
            "p_value": result["p_value"],
            "effect_size": result["eta_squared"]
        }
    
    # Best performers
    strategy_means = df.groupby("strategy").agg({
        "bias_reduction_percentage": "mean",
        "fluency_score": "mean", 
        "bleu_4_score": "mean",
        "is_gender_neutral": "mean"
    })
    
    findings["best_performers"] = {
        "bias_reduction": strategy_means["bias_reduction_percentage"].idxmax(),
        "fluency": strategy_means["fluency_score"].idxmax(),
        "meaning_preservation": strategy_means["bleu_4_score"].idxmax(),
        "neutralization_success": strategy_means["is_gender_neutral"].idxmax()
    }
    
    return findings

if 'df' in locals() and 'anova_results' in locals():
    findings = generate_findings_summary(df, anova_results)
    
    print("🔍 KEY FINDINGS SUMMARY")
    print("=" * 60)
    
    # Best performing strategies
    print(f"\n🏆 BEST PERFORMING STRATEGIES:")
    for metric, strategy in findings["best_performers"].items():
        print(f"  {metric.replace('_', ' ').title()}: {strategy}")
    
    # Statistical significance
    print(f"\n📊 STATISTICAL SIGNIFICANCE:")
    for metric, result in findings["statistical_significance"].items():
        significance = "✅ Significant" if result["significant"] else "❌ Not significant"
        effect = "Large" if result["effect_size"] > 0.14 else "Medium" if result["effect_size"] > 0.06 else "Small"
        print(f"  {metric.replace('_', ' ').title()}: {significance} (p={result['p_value']:.4f}, Effect: {effect})")
    
    # Strategy performance details
    print(f"\n📈 DETAILED STRATEGY PERFORMANCE:")
    for strategy, performance in findings["strategy_performance"].items():
        print(f"\n  {strategy.upper()}:")
        print(f"    Bias Reduction: {performance['bias_reduction']['mean']:.1f}% (±{performance['bias_reduction']['std']:.1f})")
        print(f"    Fluency Score: {performance['fluency']['mean']:.3f} (±{performance['fluency']['std']:.3f})")
        print(f"    BLEU-4 Score: {performance['meaning_preservation']['mean']:.3f} (±{performance['meaning_preservation']['std']:.3f})")
        print(f"    Success Rate: {performance['neutralization_success_rate']:.1%}")
    
    print(f"\n✅ Findings summary complete!")

## 6. Export and Reporting

Exporting results for academic presentation and publication.

In [ ]:
# Export results for academic presentation
def export_academic_results(df, anova_results, findings, output_dir):
    """Export results in formats suitable for academic presentation"""
    
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    exports = []
    
    # 1. Summary statistics table (CSV)
    summary_stats = df.groupby("strategy").agg({
        "bias_reduction_percentage": ["mean", "std", "count"],
        "fluency_score": ["mean", "std"],
        "bleu_4_score": ["mean", "std"],
        "semantic_similarity": ["mean", "std"],
        "is_gender_neutral": ["mean"]
    }).round(3)
    
    summary_stats.columns = ['_'.join(col).strip() for col in summary_stats.columns]
    summary_file = output_path / "summary_statistics.csv"
    summary_stats.to_csv(summary_file)
    exports.append(summary_file)
    
    # 2. ANOVA results (JSON)
    anova_file = output_path / "anova_results.json"
    with open(anova_file, 'w') as f:
        # Convert numpy types to native Python types for JSON serialization
        anova_export = {}
        for metric, result in anova_results.items():
            anova_export[metric] = {
                "f_statistic": float(result["f_statistic"]),
                "p_value": float(result["p_value"]),
                "eta_squared": float(result["eta_squared"]),
                "significant": bool(result["significant"])
            }
        json.dump(anova_export, f, indent=2)
    exports.append(anova_file)
    
    # 3. Complete dataset (CSV)
    dataset_file = output_path / "complete_dataset.csv"
    df.to_csv(dataset_file, index=False)
    exports.append(dataset_file)
    
    # 4. Key findings report (Markdown)
    findings_file = output_path / "key_findings.md"
    with open(findings_file, 'w') as f:
        f.write("# Gender Bias in LLMs - Key Findings\\n\\n")
        
        f.write("## Best Performing Strategies\\n\\n")
        for metric, strategy in findings["best_performers"].items():
            f.write(f"- **{metric.replace('_', ' ').title()}**: {strategy}\\n")
        
        f.write("\\n## Statistical Significance\\n\\n")
        for metric, result in findings["statistical_significance"].items():
            significance = "Significant" if result["significant"] else "Not significant"
            effect = "Large" if result["effect_size"] > 0.14 else "Medium" if result["effect_size"] > 0.06 else "Small"
            f.write(f"- **{metric.replace('_', ' ').title()}**: {significance} (p={result['p_value']:.4f}, Effect size: {effect})\\n")
        
        f.write("\\n## Strategy Performance Summary\\n\\n")
        for strategy, performance in findings["strategy_performance"].items():
            f.write(f"### {strategy.upper()}\\n")
            f.write(f"- Bias Reduction: {performance['bias_reduction']['mean']:.1f}% (±{performance['bias_reduction']['std']:.1f})\\n")
            f.write(f"- Fluency Score: {performance['fluency']['mean']:.3f} (±{performance['fluency']['std']:.3f})\\n")
            f.write(f"- BLEU-4 Score: {performance['meaning_preservation']['mean']:.3f} (±{performance['meaning_preservation']['std']:.3f})\\n")
            f.write(f"- Success Rate: {performance['neutralization_success_rate']:.1%}\\n\\n")
    
    exports.append(findings_file)
    
    return exports

# Export results
if 'df' in locals() and 'anova_results' in locals() and 'findings' in locals():
    export_dir = project_root / "data" / "results" / "academic_export"
    
    print("📤 Exporting results for academic presentation...")
    exported_files = export_academic_results(df, anova_results, findings, export_dir)
    
    print(f"✅ Exported {len(exported_files)} files:")
    for file_path in exported_files:
        print(f"  📄 {file_path.name}")
    
    print(f"\\n📁 All files saved to: {export_dir}")
    
    # Create a citation-ready summary
    print(f"\\n📝 CITATION-READY SUMMARY:")
    print(f"   Study: Gender Bias in Large Language Models")
    print(f"   Total experiments: {len(df)}")
    print(f"   Strategies tested: {df['strategy'].nunique()}")
    print(f"   Models evaluated: {df['model'].nunique()}")
    print(f"   Paragraphs analyzed: {df['paragraph_id'].nunique()}")
    
    significant_metrics = [m for m, r in anova_results.items() if r['significant']]
    print(f"   Significant differences found in: {', '.join(significant_metrics) if significant_metrics else 'No metrics'}")
    
    best_overall = df.groupby("strategy")["bias_reduction_percentage"].mean().idxmax()
    best_score = df.groupby("strategy")["bias_reduction_percentage"].mean().max()
    print(f"   Best performing strategy: {best_overall} ({best_score:.1f}% bias reduction)")

else:
    print("⚠️ Cannot export - missing analysis results. Please run the analysis cells first.")

## 7. Next Steps and Recommendations

### For Academic Presentation:

1. **Statistical Reporting**: Use the ANOVA results and effect sizes in your methodology section
2. **Visualization**: Include the generated plots in your presentation/paper
3. **Discussion Points**: Focus on the trade-offs between bias reduction and text quality
4. **Limitations**: Discuss the scope of gendered terms detected and potential improvements

### For Further Research:

1. **Extended Corpus**: Test with more paragraphs and diverse domains
2. **Additional Metrics**: Include human evaluation for bias detection
3. **Cross-Model Analysis**: Compare performance across different LLM architectures
4. **Temporal Analysis**: Study consistency across multiple runs

### Files Generated:

- **Summary Statistics**: For methodology and results sections
- **ANOVA Results**: For statistical significance reporting
- **Complete Dataset**: For reproducibility and peer review
- **Key Findings**: For discussion and conclusion sections

---

**Run all cells in sequence to reproduce the complete analysis!** 🚀